<a href="https://colab.research.google.com/github/Kezia-Hector/Exchange_Coefficient/blob/main/All_Basin_Hurricanes_Data_Collection_Upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install netCDF4
!pip install scipy
import netCDF4 as nc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import netCDF4 as nc
import numpy as np

In [ ]:
url = 'url: https://cds.climate.copernicus.eu/api/v2'

key = #insert key here

with open('/root/.cdsapirc', 'w') as f:
    f.write('\n'.join([url, key]))

with open('/root/.cdsapirc') as f:
    print(f.read())

!pip install cdsapi

url: https://cds.climate.copernicus.eu/api/v2
key: 131673:6958e515-b1c8-4e77-b4e2-3f0359c337d4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cdsapi: filename=cdsapi-0.5.1-py2.py3-none-any.whl size=11699 sha256=4f740727445da44a47b3c78dac02014650e5df81e129596845ba2717262d8eae
  Stored in directory: /root/.cache/pip/wheels/1d/2e/3c/3746e0cd076320584d6f47b688da7b13a5d5d1a92606779ca4
Successfully built cdsapi


In [ ]:
TC_data_dir = '/content/drive/MyDrive/Colab Notebooks/tcdata_netcdf/'#Shared with me/MCFC IFs/Value Chain Resilience/Resiliency UROP/Kezia'
basins = ['shtracks','wptracks','attracks','iotracks','eptracks']

In [ ]:
errorstorms = []

In [ ]:
for basin in basins:
  fn = TC_data_dir+basin+'.nc'
  ds = nc.Dataset(fn)
  # get list of storm names
  sn_data= ds.variables['stnamec'][:]
  N_chars_per_name = sn_data.shape[0]
  N_dat_pts = sn_data.shape[1]
  storm_names = []
  for i in range(N_dat_pts):
    name_bytes = sn_data.transpose()[i]
    name_string = name_bytes.tobytes().decode("utf-8")
    storm_names.append(name_string)
  #print(storm_names)
      # see what variables are there
      #print(ds.variables.keys())
      # look at a specific variable
      # get the data out of the variable
  lat = ds.variables['latmc'][:]
  lon= ds.variables['longmc'][:] # longitude
  month= ds.variables['monthmc'][:] # month
  day= ds.variables['daymc'][:] # day
  hour= ds.variables['hourmc'][:] # hour
  v= ds.variables['vsmc'][:] # speed m/s
  year = ds.variables["yearic"][:]
  #Database only has from 1959 forward so change the startpt to 1959 rather than the beginning of the dataset
  startpt = np.argwhere(np.array(year).transpose() >= 1959)
  updated_storm_names = storm_names[startpt[0][0]:]
  #Storms we have already looked at
  Done_Storm_Names = []
  if 'Not_named' in storm_names[0]:
    Done_Storm_Names.append(storm_names[0])

  for char in updated_storm_names:
    if char not in Done_Storm_Names:
      match = [i for i, s in enumerate(updated_storm_names) if char in s]
      n = len(match)
      i=0
      Done_Storm_Names.append(char)
      while i< n:
        Storm_number = match[i]
        Storm_lat_array = np.array(lat).transpose()[Storm_number + startpt[0][0]][:]
        Storm_lon_array = np.array(lon).transpose()[Storm_number + startpt[0][0]][:]
        Storm_month_array = np.array(month).transpose()[Storm_number + startpt[0][0]][:]
        Storm_day_array = np.array(day).transpose()[Storm_number + startpt[0][0]][:]
        Storm_hour_array = np.array(hour).transpose()[Storm_number + startpt[0][0]][:]
        Storm_v_array = np.array(v).transpose()[Storm_number + startpt[0][0]][:]
        Storm_year = np.array(year).transpose()[Storm_number + startpt[0][0]]
        if Storm_year < 1959.0:
          i+=1
          continue
        else:
          max_v = max(Storm_v_array)
          max_v_location = np.argwhere(Storm_v_array == max_v)[0][0]
          max_v_hour = Storm_hour_array[max_v_location]
          max_v_day = Storm_day_array[max_v_location]
          max_v_month = Storm_month_array[max_v_location]
          if max_v_hour < 10:
            max_v_hour = '0'+str(int(max_v_hour))+':00'
          elif max_v_hour >= 10:
            max_v_hour = str(int(max_v_hour))+':00'
          if max_v_day < 10:
            max_v_day = '0' + str(int(max_v_day))
          elif max_v_day >= 10:
            max_v_day = str(int(max_v_day))
          if max_v_month < 10:
            max_v_month = '0' + str(int(max_v_month))
          elif max_v_month >= 10:
            max_v_month = str(int(max_v_month))
          try:
            import cdsapi
            c = cdsapi.Client()
            c.retrieve(
            'reanalysis-era5-single-levels',
              {
                'product_type': 'reanalysis',
                'variable': [
                    '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                    '2m_temperature', 'sea_surface_temperature', 'surface_pressure',
                ],
                'year': '{0}'.format(int(Storm_year)),
                'month': '{0}'.format(max_v_month),
                'day': '{0}'.format(max_v_day),
                'time': '{0}'.format(max_v_hour),
                'format': 'netcdf',
                
              },
              '/content/drive/MyDrive/Colab Notebooks/{0} Hurricane Data/{1}EverythingElse.nc'.format(basin,char+str(Storm_year)))
      
            c.retrieve( 
            'reanalysis-era5-pressure-levels',
              {
                'product_type': 'reanalysis',
                'variable': 'temperature',
                'pressure_level': '100',
                'year': '{0}'.format(int(Storm_year)),
                'month': '{0}'.format(max_v_month),
                'day': '{0}'.format(max_v_day),
                'time': '{0}'.format(max_v_hour),
                'format': 'netcdf',
              },
            '/content/drive/MyDrive/Colab Notebooks/{0} Hurricane Data/{1}Pressure.nc'.format(basin,char+str(Storm_year)))
          
          except:
            errorstorms.append(basin+char+str(Storm_year))
          finally:
            i+=1

In [ ]:
print(errorstorms)